# Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data 

In [2]:
phase_behaviour = pd.read_csv("Data\Phase behaviour (5 pc step).csv")
raw_phase_behaviour = pd.read_csv("Data\Phase boundaries.csv")

In [1]:
# Number of data points per phase diagram
pb_points = 94 

# Create figure with subplots
columns = 4
rows = int(np.ceil(phase_behaviour.shape[0]/(pb_points*columns)))

sub_specs = list()
for j in range(columns):
    sub_specs.append({'type': 'scatterternary'})

specs = list()
for i in range(rows):
    specs.append(sub_specs)

fig = make_subplots(
    rows = rows, 
    cols = columns,
    specs = specs,
    vertical_spacing=0.05
)

# Define investigated area
area = np.array((0.4, 0.25, 0.4, 0.6, 0.375, 0, 0, 0.375, 0.6)).reshape(3,3)

# Define colour map
color_map = {
    "Microemulsion": "green",
    "Non-microemulsion": "red",
    "Boundary" : "grey"
}

# Plot each phase diagram
for i in range(int(phase_behaviour.shape[0]/pb_points)):

    # Helper values
    phase_diagram = i  # Phase diagram number
    j = phase_diagram*pb_points  # Starting point for each phase diagram dataset

    PS = phase_behaviour['PS'][j] # Polar solvent (PS) name abbreviation
    NPS = phase_behaviour['NPS'][j] # Non-polar solvent (NPS) name abbreviation

    row = int(np.floor(phase_diagram/columns)+1)
    column = np.remainder(phase_diagram,columns)+1
    
    for phase_behaviour_type, colour in color_map.items():
        # Get separate data series for microemulsions and non-micoremulsions and plot on ternary scatter plots
        if phase_behaviour_type == 'Microemulsion' or phase_behaviour_type == 'Non-microemulsion':
            SPC_conc = phase_behaviour[j:j+pb_points].loc[phase_behaviour['Phase behaviour'] == phase_behaviour_type, 'SPC concentration']
            NPS_conc = phase_behaviour[j:j+pb_points].loc[phase_behaviour['Phase behaviour'] == phase_behaviour_type, 'NPS concentration']
            PS_conc = phase_behaviour[j:j+pb_points].loc[phase_behaviour['Phase behaviour'] == phase_behaviour_type, 'PS concentration']
            
            fig.add_trace(go.Scatterternary({'mode' :'markers', 'a' : SPC_conc, 'b' : NPS_conc, 'c' : PS_conc,
                'marker_symbol': 'square', 
                'marker_size' :7,
                'marker_color': colour, 
                'marker_opacity' :0.75,
                'legendgroup' :phase_behaviour_type,  
                'name' :phase_behaviour_type,
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
        
        # Get separate data series for boundary data and plot on ternary scatter plots
        if phase_behaviour_type == 'Boundary':
            SPC_conc = phase_behaviour[j:j+pb_points].loc[phase_behaviour['Phase boundary'] == phase_behaviour_type, 'SPC concentration']
            NPS_conc = phase_behaviour[j:j+pb_points].loc[phase_behaviour['Phase boundary'] == phase_behaviour_type, 'NPS concentration']
            PS_conc = phase_behaviour[j:j+pb_points].loc[phase_behaviour['Phase boundary'] == phase_behaviour_type, 'PS concentration']
            
            # Plot phase boundary data
            fig.add_trace(go.Scatterternary({'mode' :'markers', 'a' : SPC_conc, 'b' : NPS_conc, 'c' : PS_conc,
                'marker_symbol': 'square-dot', 
                'marker_size' :8,
                'marker_line_color': 'black',
                'marker_line_width': 2,
                'marker_color': colour, 
                'marker_opacity' :0.25,
                'legendgroup' :phase_behaviour_type,  
                'name' :phase_behaviour_type, 
                'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}), 
                row = row, col = column)
        
            # Plot investigated area boundary
            fig.add_trace(go.Scatterternary({'mode':'lines', 
            'a': area[0],
            'b': area[1],
            'c': area[2],
            'line_color': '#aaaaaa',
            'line_width': 1, 
            'name': 'Boundary of investigated area',
            'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}),
            row = row, col = column)
        
        # Update axis names
        fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                            baxis_title = str(NPS + ' (w/w)'), 
                            caxis_title = str(PS + ' (w/w)'), 
                            row = row, col = column)
    
    # Get corresponding phase boundary data
    for k in range(raw_phase_behaviour.shape[0]):
        SPC_conc = raw_phase_behaviour.loc[((raw_phase_behaviour['NPS'] == NPS) & (raw_phase_behaviour['PS'] == PS)), 'SPC concentration']
        NPS_conc = raw_phase_behaviour.loc[((raw_phase_behaviour['NPS'] == NPS) & (raw_phase_behaviour['PS'] == PS)), 'NPS concentration']
        PS_conc = raw_phase_behaviour.loc[((raw_phase_behaviour['NPS'] == NPS) & (raw_phase_behaviour['PS'] == PS)), 'PS concentration']
        
    fig.add_trace(go.Scatterternary({'mode' :'lines', 'a' : SPC_conc, 'b' : NPS_conc, 'c' : PS_conc,
        'line_color': 'black',
        'line_width': 1, 
        'name': 'Boundary of investigated area',
        'hovertemplate':'PS: %{c}<br>SPC: %{a}<br> NPS: %{b}'}),
        row = row, col = column)

    # Update axis names
    fig.update_ternaries(aaxis_title = 'SPC (w/w)', 
                        baxis_title = str(NPS + ' (w/w)'), 
                        caxis_title = str(PS + ' (w/w)'), 
                        row = row, col = column)
    
# Update all plots formating
fig.update_ternaries(
    aaxis_dtick=0.1, baxis_dtick = 0.1, caxis_dtick = 0.1,
    aaxis_linecolor='#aaaaaa', baxis_linecolor='#aaaaaa', caxis_linecolor='#aaaaaa',
    aaxis_gridcolor='#cccccc', baxis_gridcolor='#cccccc', caxis_gridcolor='#cccccc',
    aaxis_tickfont_size=8, baxis_tickfont_size=8, caxis_tickfont_size=8, 
    bgcolor = 'white')

fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1500,
    height=2000)


fig.show()